# IMPORTS

In [1]:
from __future__ import print_function
import random
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchinfo import summary
from tqdm import tqdm
from model import MnistNet_2


def seed_everything(seed=10):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything(1)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

# MODEL

In [2]:
model = MnistNet_2()
summary(model, input_size=(1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
MnistNet_4                               [1, 10]                   --
├─Sequential: 1-1                        [1, 8, 12, 12]            --
│    └─Conv2d: 2-1                       [1, 8, 26, 26]            72
│    └─ReLU: 2-2                         [1, 8, 26, 26]            --
│    └─BatchNorm2d: 2-3                  [1, 8, 26, 26]            16
│    └─Dropout: 2-4                      [1, 8, 26, 26]            --
│    └─Conv2d: 2-5                       [1, 16, 24, 24]           1,152
│    └─ReLU: 2-6                         [1, 16, 24, 24]           --
│    └─BatchNorm2d: 2-7                  [1, 16, 24, 24]           32
│    └─Dropout: 2-8                      [1, 16, 24, 24]           --
│    └─Conv2d: 2-9                       [1, 8, 24, 24]            128
│    └─MaxPool2d: 2-10                   [1, 8, 12, 12]            --
├─Sequential: 1-2                        [1, 16, 6, 6]             --
│    └─Conv

# DATALOADER

In [3]:
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.RandomAffine(degrees=15),  # ,  scale=(0.95, 1.05)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                    batch_size=batch_size, shuffle=True, **kwargs)


# TRAINING & TESTING

In [4]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'Epoch={epoch} Batch={batch_idx} loss={loss.item():.7f} Accuracy={100. * correct / len(train_loader.dataset):.2f}%')


def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.7f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# RUN

In [5]:
model = MnistNet_2().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.075,momentum=0.9, nesterov=True)
scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=2, gamma=0.65, verbose=True)

criterion = nn.NLLLoss()

for epoch in range(15):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)
    scheduler.step()

Adjusting learning rate of group 0 to 7.5000e-02.


Epoch=0 Batch=468 loss=0.1479667 Accuracy=92.80%: 100%|██████████| 469/469 [00:10<00:00, 45.39it/s]



Test set: Average loss: 0.0003707, Accuracy: 9855/10000 (98.55%)

Adjusting learning rate of group 0 to 7.5000e-02.


Epoch=1 Batch=468 loss=0.0885876 Accuracy=97.39%: 100%|██████████| 469/469 [00:10<00:00, 44.58it/s]



Test set: Average loss: 0.0002947, Accuracy: 9886/10000 (98.86%)

Adjusting learning rate of group 0 to 4.8750e-02.


...


Test set: Average loss: 0.0001575, Accuracy: 9947/10000 (99.47%)

Adjusting learning rate of group 0 to 5.6564e-03.


Epoch=13 Batch=468 loss=0.0625362 Accuracy=98.87%: 100%|██████████| 469/469 [00:10<00:00, 45.20it/s]



Test set: Average loss: 0.0001386, Accuracy: 9940/10000 (99.40%)

Adjusting learning rate of group 0 to 3.6767e-03.


Epoch=14 Batch=468 loss=0.0151680 Accuracy=98.85%: 100%|██████████| 469/469 [00:10<00:00, 45.68it/s]



Test set: Average loss: 0.0001644, Accuracy: 9934/10000 (99.34%)

Adjusting learning rate of group 0 to 3.6767e-03.


# Target
- Parameters < 8K
- Batch normalization and regularization

# Result
- Parameters: 7,416
- Best Training Accuracy: 98.87%
- Best Test Accuracy: 99.47%

# Analysis
- Test accuracy reached 99.47% but consistency is not good enough
- Train-test gap can be seen as train accuracy doesn't reach 99%
- Can try to reduce parameters futher below 7k and tweak around dropout rate a bit more